In [1]:
import IPython
from IPython import display 
print("IPython version: {}". format(IPython.__version__))
import keras as k
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, Callback
from keras.layers import Dense, Dropout
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model

import matplotlib 
print("matplotlib version: {}". format(matplotlib.__version__))
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np 
print("NumPy version: {}". format(np.__version__))


import os
import os.path

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import random
from random import seed

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

from sklearn.preprocessing import StandardScaler, MinMaxScaler


import sys 
print("Python version: {}". format(sys.version))

import tensorflow as tf
print('TensorFlow: {}'.format(tf.__version__))

import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
print(check_output(["ls", "/home/willian/Keras_Litologia/arquivos_csv"]).decode("utf8"))

IPython version: 7.8.0


Using TensorFlow backend.


matplotlib version: 3.1.1
NumPy version: 1.16.5
pandas version: 0.25.1
seaborn version: 0.9.0
Python version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
TensorFlow: 1.14.0
test.csv
train.csv



In [2]:
np.random.seed(7)

## Preprocessamento

In [3]:
def load_data():
    """
    Função para carregar o datasets pelo panda.
    Retun: a variável passa a ser um dataframe do pandas, podendo ter acesso aos métodos da lib.
        dftrain = treinamento
        dftest = teste
    """
    dftrain = pd.read_csv('/home/willian/Keras_Litologia/arquivos_csv/train.csv')
    dftest = pd.read_csv('/home/willian/Keras_Litologia/arquivos_csv/test.csv')
    return dftrain, dftest

In [4]:
def data_encode(data_frame, classificador):
    """
    Função para transformar a coluna do classificado em multiclassificadores *litologia de 1 a 7.
    data_frame: dicionário a ser recebido.
    classificador: nome da coluna do classificador.
    Return: Um novo data frame com as linhas de classificação transformada em colunas.        
    """
    data_frame = pd.get_dummies(data_frame, columns=[classificador])
    return data_frame

In [5]:
def variables_x_y(data_frame, cabecalho, n):
    """
    Função para transforma as variáveis de características e classificadores.
    n  = as colunas que fazem parte do classificador.
    y = classificador.
    x = características.
    return: um data_frame de características e outro de classificador do dataset.
    """
    x = data_frame[cabecalho[:n]]
    y = data_frame[cabecalho[n:]]
    return x, y

In [6]:
def transfer_array_np(data_frame, vtype):
    """
    Função para transformar o data_frame em array numpy [ matriz numpy]
    df = data_frame a ser transformado
    vtype = nome da variavel do tipo a ser transformada
    Return: matriz numpy do dataset.
    """
    np_data_frame = np.array(data_frame, dtype=vtype)
    return np_data_frame

In [7]:
def preprocessamento():
    dftrain, dftest = load_data()
    train = data_encode(dftrain, 'Litologia')
    test = data_encode(dftest, 'Litologia')
    train = train.drop(['Poço'], axis = 1)
    test = test.drop(['Poço'], axis = 1)
    cabecalho_list = list(train.columns.values)
    
    # n = - 7 representa primeira coluna do classificado.
    n = -7
    
    x_train, y_train = variables_x_y(train, cabecalho_list, n)
    x_test, y_test = variables_x_y(test, cabecalho_list, n)
    X_train = transfer_array_np(x_train, 'float32')
    y_train = transfer_array_np(y_train, 'int8')
    X_test = transfer_array_np(x_test, 'float32')
    y_test = transfer_array_np(y_test, 'int8')
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(X_train)
    x_test = scaler.fit_transform(X_test)
    return x_train, y_train, x_test, y_test

In [8]:
x_train, y_train, x_test, y_test = preprocessamento()

## ANN - Keras Modelo

In [9]:
def create_model(n_input, n_hidden, n_output, func_input, func_hidden, func_output):
    model = Sequential()
    model.add(Dense(n_input, input_dim = n_input, activation = func_input, kernel_initializer = 'uniform', bias_initializer = 'zeros'))
    model.add(Dense(n_hidden, activation = func_hidden))
    model.add(Dense(n_output, activation = func_output))
    opt = SGD(lr = 0.01, momentum = 0.9)
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc', 'mse'])    
    print('Created to model.')
    return model

In [10]:
def load_model(path_file, num_model):
    json_file = open(path_file + 'model_' + num_model + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    print('Loaded model from disk')
    return model

In [11]:
epoch = 2500
batch_s = 10
path_file_models = '/home/willian/Keras_Litologia/model_save/Teste'

In [12]:
def verific_load_or_create(n_input, n_hidden, n_output, func_input, func_hidden, func_output, num_model):
    arquivo = True
    f_path = path_file_models + num_model + '/'
    file = f_path + 'model_' + num_model + '.json'
    if os.path.isfile(file) == True:
        model = load_model(path_file_models, num_model)
        return model, arquivo
    else:
        model = create_model(n_input, n_hidden, n_output, 'relu', 'relu', 'softmax')
        model.summary()
        #save_model(path_file_models, model16, '16')
        arquivo = False
        return model, arquivo 

In [13]:
def history_model(model, arquivo, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, num_model):
    if arquivo == False:
        checkpoint = [ModelCheckpoint(path_file_models + num_model + '/' + '{epoch:02d}_{acc:.2f}' + num_model + ".json", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)]
        history = model.fit(x_train, y_train, validation_data = (x_test, y_test), callbacks=checkpoint, nb_epoch = epoch, verbose = 1, batch_size=batch_s, shuffle=True)
        score_train, score_test = score_model(model, x_train, y_train, x_test, y_test)
        return history, score_train, score_test
    else:
        opt = SGD(lr = 0.01, momentum = 0.9)
        history = model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc', 'mse'])
        score_trapath_file_modelsin, score_test = score_model(model, x_train, y_train, x_test, y_test)
        return history, score_train, score_test 

In [14]:
def score_model(model, x_train, y_train, x_test, y_test):
    score_train = model.evaluate(x_train, y_train, verbose = 0)
    score_test = model.evaluate(x_test, y_test, verbose = 0)
    return score_train, score_test

In [15]:
def display_score(score_train, score_test):
    print(" Baseline Error Train: %.2f%%" % (100-score_train[1]*100))
    print(' Train Accuracy:', score_train[1])
    print(" Baseline Error Test: %.2f%%" % (100-score_test[1]*100))
    print(' Test Accuracy:', score_test[1])

### 1 - Camada Intermediária (Hidden-Layers) : quantidade de neurônio

#### Obs: Manipulação apenas da quantidade de neurônios na camada, demais parâmetros fixos.

#### 1.1 Teste da Camada Oculta (Hidden Layer) = 5

In [16]:
model11, arquivo11 = verific_load_or_create(20, 5, 7, 'relu', 'relu', 'softmax', '12')

W1101 16:17:52.073290 140127520225088 deprecation_wrapper.py:119] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1101 16:17:52.092795 140127520225088 deprecation_wrapper.py:119] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1101 16:17:52.097028 140127520225088 deprecation_wrapper.py:119] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1101 16:17:52.187721 140127520225088 deprecation_wrapper.py:119] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Opt

Created to model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 42        
Total params: 567
Trainable params: 567
Non-trainable params: 0
_________________________________________________________________


In [17]:
history11, score_train11, score_test11 = history_model(model11, arquivo11, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, '11')

W1101 16:17:54.930513 140127520225088 deprecation.py:323] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1101 16:17:55.019214 140127520225088 deprecation_wrapper.py:119] From /home/willian/.conda/envs/conda_tensorflow/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 46 samples, validate on 7 samples
Epoch 1/2500
46/46 [==============================] - 1s 24ms/step - loss: 1.9426 - acc: 0.1957 - mean_squared_error: 0.1223 - val_loss: 1.9462 - val_acc: 0.1429 - val_mean_squared_error: 0.1225

Epoch 00001: acc improved from -inf to 0.19565, saving model to /home/willian/Keras_Litologia/model_save/Teste11/01_0.2011.json
Epoch 2/2500
46/46 [==============================] - 0s 331us/step - loss: 1.9251 - acc: 0.1957 - mean_squared_error: 0.1216 - val_loss: 1.9401 - val_acc: 0.1429 - val_mean_squared_error: 0.1222

Epoch 00002: acc did not improve from 0.19565
Epoch 3/2500
46/46 [==============================] - 0s 786us/step - loss: 1.9016 - acc: 0.1957 - mean_squared_error: 0.1207 - val_loss: 1.9329 - val_acc: 0.1429 - val_mean_squared_error: 0.1219

Epoch 00003: acc did not improve from 0.19565
Epoch 4/2500
46/46 [==============================] - 0s 922us/step - loss: 1.8798 - acc: 0.1957 - mean_squared_error: 0.1198 - val_loss: 1.9307 - 

46/46 [==============================] - 0s 759us/step - loss: 1.1769 - acc: 0.5870 - mean_squared_error: 0.0886 - val_loss: 1.4207 - val_acc: 0.5714 - val_mean_squared_error: 0.0992

Epoch 00028: acc improved from 0.56522 to 0.58696, saving model to /home/willian/Keras_Litologia/model_save/Teste11/28_0.5911.json
Epoch 29/2500
46/46 [==============================] - 0s 750us/step - loss: 1.1406 - acc: 0.5870 - mean_squared_error: 0.0861 - val_loss: 1.3924 - val_acc: 0.5714 - val_mean_squared_error: 0.0968

Epoch 00029: acc did not improve from 0.58696
Epoch 30/2500
46/46 [==============================] - 0s 776us/step - loss: 1.1081 - acc: 0.6087 - mean_squared_error: 0.0841 - val_loss: 1.3495 - val_acc: 0.5714 - val_mean_squared_error: 0.0930

Epoch 00030: acc improved from 0.58696 to 0.60870, saving model to /home/willian/Keras_Litologia/model_save/Teste11/30_0.6111.json
Epoch 31/2500
46/46 [==============================] - 0s 1ms/step - loss: 1.0953 - acc: 0.5652 - mean_squared_e

46/46 [==============================] - 0s 682us/step - loss: 0.7456 - acc: 0.6739 - mean_squared_error: 0.0614 - val_loss: 0.9352 - val_acc: 0.4286 - val_mean_squared_error: 0.0705

Epoch 00059: acc did not improve from 0.67391
Epoch 60/2500
46/46 [==============================] - 0s 656us/step - loss: 0.7496 - acc: 0.6304 - mean_squared_error: 0.0611 - val_loss: 0.9171 - val_acc: 0.7143 - val_mean_squared_error: 0.0689

Epoch 00060: acc did not improve from 0.67391
Epoch 61/2500
46/46 [==============================] - 0s 612us/step - loss: 0.7386 - acc: 0.6957 - mean_squared_error: 0.0605 - val_loss: 1.0012 - val_acc: 0.4286 - val_mean_squared_error: 0.0764

Epoch 00061: acc improved from 0.67391 to 0.69565, saving model to /home/willian/Keras_Litologia/model_save/Teste11/61_0.7011.json
Epoch 62/2500
46/46 [==============================] - 0s 796us/step - loss: 0.7480 - acc: 0.6522 - mean_squared_error: 0.0609 - val_loss: 0.9761 - val_acc: 0.4286 - val_mean_squared_error: 0.0736



Epoch 00089: acc did not improve from 0.82609
Epoch 90/2500
46/46 [==============================] - 0s 543us/step - loss: 0.5296 - acc: 0.7826 - mean_squared_error: 0.0412 - val_loss: 0.6584 - val_acc: 0.8571 - val_mean_squared_error: 0.0452

Epoch 00090: acc did not improve from 0.82609
Epoch 91/2500
46/46 [==============================] - 0s 485us/step - loss: 0.5111 - acc: 0.8261 - mean_squared_error: 0.0397 - val_loss: 0.6980 - val_acc: 0.7143 - val_mean_squared_error: 0.0525

Epoch 00091: acc did not improve from 0.82609
Epoch 92/2500
46/46 [==============================] - 0s 779us/step - loss: 0.5233 - acc: 0.8043 - mean_squared_error: 0.0412 - val_loss: 0.6491 - val_acc: 0.7143 - val_mean_squared_error: 0.0476

Epoch 00092: acc did not improve from 0.82609
Epoch 93/2500
46/46 [==============================] - 0s 818us/step - loss: 0.5330 - acc: 0.7826 - mean_squared_error: 0.0425 - val_loss: 0.6226 - val_acc: 0.8571 - val_mean_squared_error: 0.0404

Epoch 00093: acc did no

...


Epoch 02341: acc did not improve from 1.00000
Epoch 2342/2500
46/46 [==============================] - 0s 707us/step - loss: 0.0028 - acc: 1.0000 - mean_squared_error: 1.2169e-05 - val_loss: 2.7318 - val_acc: 0.7143 - val_mean_squared_error: 0.0750

Epoch 02342: acc did not improve from 1.00000
Epoch 2343/2500
46/46 [==============================] - 0s 579us/step - loss: 0.0029 - acc: 1.0000 - mean_squared_error: 1.3242e-05 - val_loss: 2.7295 - val_acc: 0.7143 - val_mean_squared_error: 0.0749

Epoch 02343: acc did not improve from 1.00000
Epoch 2344/2500
46/46 [==============================] - 0s 539us/step - loss: 0.0029 - acc: 1.0000 - mean_squared_error: 1.3497e-05 - val_loss: 2.7368 - val_acc: 0.7143 - val_mean_squared_error: 0.0752

Epoch 02344: acc did not improve from 1.00000
Epoch 2345/2500
46/46 [==============================] - 0s 820us/step - loss: 0.0029 - acc: 1.0000 - mean_squared_error: 1.3017e-05 - val_loss: 2.7427 - val_acc: 0.7143 - val_mean_squared_error: 0.0755



Epoch 02374: acc did not improve from 1.00000
Epoch 2375/2500
46/46 [==============================] - 0s 680us/step - loss: 0.0027 - acc: 1.0000 - mean_squared_error: 1.2016e-05 - val_loss: 2.7731 - val_acc: 0.7143 - val_mean_squared_error: 0.0764

Epoch 02375: acc did not improve from 1.00000
Epoch 2376/2500
46/46 [==============================] - 0s 669us/step - loss: 0.0030 - acc: 1.0000 - mean_squared_error: 1.5081e-05 - val_loss: 2.7865 - val_acc: 0.7143 - val_mean_squared_error: 0.0769

Epoch 02376: acc did not improve from 1.00000
Epoch 2377/2500
46/46 [==============================] - 0s 600us/step - loss: 0.0028 - acc: 1.0000 - mean_squared_error: 1.2615e-05 - val_loss: 2.7644 - val_acc: 0.7143 - val_mean_squared_error: 0.0760

Epoch 02377: acc did not improve from 1.00000
Epoch 2378/2500
46/46 [==============================] - 0s 792us/step - loss: 0.0029 - acc: 1.0000 - mean_squared_error: 1.3666e-05 - val_loss: 2.7619 - val_acc: 0.7143 - val_mean_squared_error: 0.0759



Epoch 02407: acc did not improve from 1.00000
Epoch 2408/2500
46/46 [==============================] - 0s 839us/step - loss: 0.0027 - acc: 1.0000 - mean_squared_error: 1.1342e-05 - val_loss: 2.7822 - val_acc: 0.7143 - val_mean_squared_error: 0.0764

Epoch 02408: acc did not improve from 1.00000
Epoch 2409/2500
46/46 [==============================] - 0s 828us/step - loss: 0.0027 - acc: 1.0000 - mean_squared_error: 1.1149e-05 - val_loss: 2.7778 - val_acc: 0.7143 - val_mean_squared_error: 0.0762

Epoch 02409: acc did not improve from 1.00000
Epoch 2410/2500
46/46 [==============================] - 0s 955us/step - loss: 0.0027 - acc: 1.0000 - mean_squared_error: 1.1806e-05 - val_loss: 2.7862 - val_acc: 0.7143 - val_mean_squared_error: 0.0765

Epoch 02410: acc did not improve from 1.00000
Epoch 2411/2500
46/46 [==============================] - 0s 995us/step - loss: 0.0027 - acc: 1.0000 - mean_squared_error: 1.1826e-05 - val_loss: 2.7879 - val_acc: 0.7143 - val_mean_squared_error: 0.0765



Epoch 02440: acc did not improve from 1.00000
Epoch 2441/2500
46/46 [==============================] - 0s 774us/step - loss: 0.0026 - acc: 1.0000 - mean_squared_error: 1.0494e-05 - val_loss: 2.8010 - val_acc: 0.7143 - val_mean_squared_error: 0.0769

Epoch 02441: acc did not improve from 1.00000
Epoch 2442/2500
46/46 [==============================] - 0s 530us/step - loss: 0.0026 - acc: 1.0000 - mean_squared_error: 1.0866e-05 - val_loss: 2.7976 - val_acc: 0.7143 - val_mean_squared_error: 0.0768

Epoch 02442: acc did not improve from 1.00000
Epoch 2443/2500
46/46 [==============================] - 0s 519us/step - loss: 0.0026 - acc: 1.0000 - mean_squared_error: 1.0500e-05 - val_loss: 2.7965 - val_acc: 0.7143 - val_mean_squared_error: 0.0768

Epoch 02443: acc did not improve from 1.00000
Epoch 2444/2500
46/46 [==============================] - 0s 820us/step - loss: 0.0026 - acc: 1.0000 - mean_squared_error: 1.0783e-05 - val_loss: 2.7902 - val_acc: 0.7143 - val_mean_squared_error: 0.0765



Epoch 02473: acc did not improve from 1.00000
Epoch 2474/2500
46/46 [==============================] - 0s 596us/step - loss: 0.0025 - acc: 1.0000 - mean_squared_error: 1.0434e-05 - val_loss: 2.8013 - val_acc: 0.7143 - val_mean_squared_error: 0.0769

Epoch 02474: acc did not improve from 1.00000
Epoch 2475/2500
46/46 [==============================] - 0s 609us/step - loss: 0.0025 - acc: 1.0000 - mean_squared_error: 1.0098e-05 - val_loss: 2.7983 - val_acc: 0.7143 - val_mean_squared_error: 0.0769

Epoch 02475: acc did not improve from 1.00000
Epoch 2476/2500
46/46 [==============================] - 0s 429us/step - loss: 0.0025 - acc: 1.0000 - mean_squared_error: 9.9783e-06 - val_loss: 2.8034 - val_acc: 0.7143 - val_mean_squared_error: 0.0770

Epoch 02476: acc did not improve from 1.00000
Epoch 2477/2500
46/46 [==============================] - 0s 603us/step - loss: 0.0025 - acc: 1.0000 - mean_squared_error: 1.0270e-05 - val_loss: 2.8009 - val_acc: 0.7143 - val_mean_squared_error: 0.0770


In [18]:
display_score(score_train11, score_test11)

 Baseline Error Train: 0.00%
 Train Accuracy: 1.0
 Baseline Error Test: 28.57%
 Test Accuracy: 0.7142857313156128


In [19]:
print(check_output(["ls", "/home/willian/Keras_Litologia/model_save/Teste11"]).decode("utf8"))

01_0.2011.json
05_0.2011.json
07_0.2211.json
08_0.2411.json
09_0.2611.json
10_0.2611.json
101_0.8511.json
104_0.8511.json
12_0.2811.json
13_0.2811.json
140_0.8711.json
14_0.3011.json
145_0.8911.json
15_0.3511.json
20_0.3711.json
21_0.3911.json
212_0.9111.json
22_0.4611.json
23_0.4811.json
232_0.9311.json
24_0.5011.json
25_0.5011.json
26_0.5411.json
27_0.5711.json
28_0.5911.json
30_0.6111.json
34_0.6111.json
348_0.9311.json
35_0.6511.json
388_0.9611.json
40_0.6511.json
443_0.9611.json
455_0.9611.json
47_0.6711.json
50_0.6711.json
538_0.9811.json
61_0.7011.json
65_0.7211.json
654_0.9811.json
69_0.7411.json
70_0.7411.json
72_0.7411.json
75_0.7611.json
76_0.7811.json
80_0.8011.json
849_1.0011.json
88_0.8311.json



#### 1.2 Teste da Camada Oculta (Hidden Layer) = 10

In [20]:
model12, arquivo12 = verific_load_or_create(20, 10, 7, 'relu', 'relu', 'softmax', '12')

Created to model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 77        
Total params: 707
Trainable params: 707
Non-trainable params: 0
_________________________________________________________________


In [21]:
history12, score_train12, score_test12 = history_model(model12, arquivo12, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, '12')

Train on 46 samples, validate on 7 samples
Epoch 1/2500
46/46 [==============================] - 1s 18ms/step - loss: 1.9482 - acc: 0.1957 - mean_squared_error: 0.1225 - val_loss: 1.9438 - val_acc: 0.1429 - val_mean_squared_error: 0.1224

Epoch 00001: acc improved from -inf to 0.19565, saving model to /home/willian/Keras_Litologia/model_save/Teste12/01_0.2012.json
Epoch 2/2500
46/46 [==============================] - 0s 375us/step - loss: 1.9344 - acc: 0.2391 - mean_squared_error: 0.1220 - val_loss: 1.9379 - val_acc: 0.1429 - val_mean_squared_error: 0.1221

Epoch 00002: acc improved from 0.19565 to 0.23913, saving model to /home/willian/Keras_Litologia/model_save/Teste12/02_0.2412.json
Epoch 3/2500
46/46 [==============================] - 0s 577us/step - loss: 1.9163 - acc: 0.2391 - mean_squared_error: 0.1213 - val_loss: 1.9312 - val_acc: 0.2857 - val_mean_squared_error: 0.1218

Epoch 00003: acc did not improve from 0.23913
Epoch 4/2500
46/46 [==============================] - 0s 887us

46/46 [==============================] - 0s 671us/step - loss: 0.8555 - acc: 0.7174 - mean_squared_error: 0.0638 - val_loss: 1.0359 - val_acc: 0.7143 - val_mean_squared_error: 0.0725

Epoch 00030: acc improved from 0.67391 to 0.71739, saving model to /home/willian/Keras_Litologia/model_save/Teste12/30_0.7212.json
Epoch 31/2500
46/46 [==============================] - 0s 676us/step - loss: 0.8226 - acc: 0.7174 - mean_squared_error: 0.0617 - val_loss: 1.0268 - val_acc: 0.7143 - val_mean_squared_error: 0.0730

Epoch 00031: acc did not improve from 0.71739
Epoch 32/2500
46/46 [==============================] - 0s 667us/step - loss: 0.8000 - acc: 0.6957 - mean_squared_error: 0.0607 - val_loss: 1.0045 - val_acc: 0.7143 - val_mean_squared_error: 0.0721

Epoch 00032: acc did not improve from 0.71739
Epoch 33/2500
46/46 [==============================] - 0s 789us/step - loss: 0.7761 - acc: 0.7391 - mean_squared_error: 0.0589 - val_loss: 0.9615 - val_acc: 0.7143 - val_mean_squared_error: 0.0694


Epoch 62/2500
46/46 [==============================] - 0s 534us/step - loss: 0.4908 - acc: 0.7826 - mean_squared_error: 0.0388 - val_loss: 0.4493 - val_acc: 0.8571 - val_mean_squared_error: 0.0366

Epoch 00062: acc did not improve from 0.84783
Epoch 63/2500
46/46 [==============================] - 0s 499us/step - loss: 0.4728 - acc: 0.8261 - mean_squared_error: 0.0369 - val_loss: 0.4675 - val_acc: 0.8571 - val_mean_squared_error: 0.0387

Epoch 00063: acc did not improve from 0.84783
Epoch 64/2500
46/46 [==============================] - 0s 624us/step - loss: 0.4678 - acc: 0.7826 - mean_squared_error: 0.0375 - val_loss: 0.4745 - val_acc: 0.8571 - val_mean_squared_error: 0.0398

Epoch 00064: acc did not improve from 0.84783
Epoch 65/2500
46/46 [==============================] - 0s 931us/step - loss: 0.4622 - acc: 0.8043 - mean_squared_error: 0.0368 - val_loss: 0.4278 - val_acc: 0.8571 - val_mean_squared_error: 0.0340

Epoch 00065: acc did not improve from 0.84783
Epoch 66/2500
46/46 [===

46/46 [==============================] - 0s 588us/step - loss: 0.3849 - acc: 0.8261 - mean_squared_error: 0.0313 - val_loss: 0.2839 - val_acc: 0.8571 - val_mean_squared_error: 0.0213

Epoch 00095: acc did not improve from 0.86957
Epoch 96/2500
46/46 [==============================] - 0s 589us/step - loss: 0.3825 - acc: 0.8478 - mean_squared_error: 0.0312 - val_loss: 0.2965 - val_acc: 0.8571 - val_mean_squared_error: 0.0237

Epoch 00096: acc did not improve from 0.86957
Epoch 97/2500
46/46 [==============================] - 0s 437us/step - loss: 0.3858 - acc: 0.8043 - mean_squared_error: 0.0317 - val_loss: 0.2748 - val_acc: 0.8571 - val_mean_squared_error: 0.0204

Epoch 00097: acc did not improve from 0.86957
Epoch 98/2500
46/46 [==============================] - 0s 713us/step - loss: 0.3760 - acc: 0.8261 - mean_squared_error: 0.0306 - val_loss: 0.2854 - val_acc: 0.8571 - val_mean_squared_error: 0.0227

Epoch 00098: acc did not improve from 0.86957
Epoch 99/2500
46/46 [=================

46/46 [==============================] - 0s 1ms/step - loss: 0.3673 - acc: 0.8478 - mean_squared_error: 0.0313 - val_loss: 0.2618 - val_acc: 0.8571 - val_mean_squared_error: 0.0225

Epoch 00128: acc did not improve from 0.89130
Epoch 129/2500
46/46 [==============================] - 0s 495us/step - loss: 0.3328 - acc: 0.8261 - mean_squared_error: 0.0273 - val_loss: 0.2314 - val_acc: 0.8571 - val_mean_squared_error: 0.0186

Epoch 00129: acc did not improve from 0.89130
Epoch 130/2500
46/46 [==============================] - 0s 378us/step - loss: 0.3714 - acc: 0.8043 - mean_squared_error: 0.0319 - val_loss: 0.2174 - val_acc: 1.0000 - val_mean_squared_error: 0.0162

Epoch 00130: acc did not improve from 0.89130
Epoch 131/2500
46/46 [==============================] - 0s 801us/step - loss: 0.3420 - acc: 0.8478 - mean_squared_error: 0.0285 - val_loss: 0.2519 - val_acc: 0.8571 - val_mean_squared_error: 0.0221

Epoch 00131: acc did not improve from 0.89130
Epoch 132/2500
46/46 [===============

...

46/46 [==============================] - 0s 458us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.7323e-06 - val_loss: 1.6577 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02346: acc did not improve from 1.00000
Epoch 2347/2500
46/46 [==============================] - 0s 572us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.5995e-06 - val_loss: 1.6557 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02347: acc did not improve from 1.00000
Epoch 2348/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.8721e-06 - val_loss: 1.6535 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02348: acc did not improve from 1.00000
Epoch 2349/2500
46/46 [==============================] - 0s 868us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5433e-06 - val_loss: 1.6560 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02349: acc did not improve from 1.00000
Epoch 2350/2500
46

46/46 [==============================] - 0s 540us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5281e-06 - val_loss: 1.6623 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02379: acc did not improve from 1.00000
Epoch 2380/2500
46/46 [==============================] - 0s 490us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5150e-06 - val_loss: 1.6625 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02380: acc did not improve from 1.00000
Epoch 2381/2500
46/46 [==============================] - 0s 486us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5141e-06 - val_loss: 1.6614 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02381: acc did not improve from 1.00000
Epoch 2382/2500
46/46 [==============================] - 0s 509us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.4309e-06 - val_loss: 1.6598 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02382: acc did not improve from 1.00000
Epoch 2383/2500


46/46 [==============================] - 0s 955us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5845e-06 - val_loss: 1.6677 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02412: acc did not improve from 1.00000
Epoch 2413/2500
46/46 [==============================] - 0s 534us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.3336e-06 - val_loss: 1.6711 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02413: acc did not improve from 1.00000
Epoch 2414/2500
46/46 [==============================] - 0s 537us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5095e-06 - val_loss: 1.6705 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02414: acc did not improve from 1.00000
Epoch 2415/2500
46/46 [==============================] - 0s 724us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9251e-06 - val_loss: 1.6786 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02415: acc did not improve from 1.00000
Epoch 2416/2500


46/46 [==============================] - 0s 634us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.3836e-06 - val_loss: 1.6763 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02445: acc did not improve from 1.00000
Epoch 2446/2500
46/46 [==============================] - 0s 606us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.1898e-06 - val_loss: 1.6827 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02446: acc did not improve from 1.00000
Epoch 2447/2500
46/46 [==============================] - 0s 521us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.3329e-06 - val_loss: 1.6821 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02447: acc did not improve from 1.00000
Epoch 2448/2500
46/46 [==============================] - 0s 483us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.6506e-06 - val_loss: 1.6841 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02448: acc did not improve from 1.00000
Epoch 2449/2500


46/46 [==============================] - 0s 508us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.1484e-06 - val_loss: 1.6850 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02478: acc did not improve from 1.00000
Epoch 2479/2500
46/46 [==============================] - 0s 417us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.1370e-06 - val_loss: 1.6849 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02479: acc did not improve from 1.00000
Epoch 2480/2500
46/46 [==============================] - 0s 590us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.3612e-06 - val_loss: 1.6817 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02480: acc did not improve from 1.00000
Epoch 2481/2500
46/46 [==============================] - 0s 604us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.2979e-06 - val_loss: 1.6880 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02481: acc did not improve from 1.00000
Epoch 2482/2500


In [22]:
display_score(score_train12, score_test12)

 Baseline Error Train: 0.00%
 Train Accuracy: 1.0
 Baseline Error Test: 14.29%
 Test Accuracy: 0.8571428656578064


In [23]:
print(check_output(["ls", "/home/willian/Keras_Litologia/model_save/Teste12"]).decode("utf8"))

01_0.2012.json
02_0.2412.json
04_0.3012.json
06_0.3012.json
07_0.3312.json
08_0.3912.json
11_0.4112.json
121_0.8912.json
148_0.8912.json
160_0.9112.json
166_0.9112.json
17_0.5012.json
18_0.5712.json
20_0.5912.json
209_0.9112.json
216_0.9312.json
23_0.5912.json
235_0.9312.json
26_0.6712.json
275_0.9612.json
30_0.7212.json
309_0.9612.json
33_0.7412.json
339_0.9812.json
34_0.7412.json
456_1.0012.json
52_0.7612.json
53_0.7812.json
61_0.8512.json
91_0.8712.json



#### 1.3 Teste da Camada Oculta (Hidden Layer) = 15

In [24]:
model13, arquivo13 = verific_load_or_create(20, 15, 7, 'relu', 'relu', 'softmax', '13')

Created to model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_8 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 112       
Total params: 847
Trainable params: 847
Non-trainable params: 0
_________________________________________________________________


In [25]:
history13, score_train13, score_test13 = history_model(model13, arquivo13, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, '13')

Train on 46 samples, validate on 7 samples
Epoch 1/2500
46/46 [==============================] - 1s 18ms/step - loss: 1.9346 - acc: 0.3043 - mean_squared_error: 0.1220 - val_loss: 1.9466 - val_acc: 0.1429 - val_mean_squared_error: 0.1225

Epoch 00001: acc improved from -inf to 0.30435, saving model to /home/willian/Keras_Litologia/model_save/Teste13/01_0.3013.json
Epoch 2/2500
46/46 [==============================] - 0s 635us/step - loss: 1.9023 - acc: 0.2391 - mean_squared_error: 0.1206 - val_loss: 1.9519 - val_acc: 0.1429 - val_mean_squared_error: 0.1226

Epoch 00002: acc did not improve from 0.30435
Epoch 3/2500
46/46 [==============================] - 0s 858us/step - loss: 1.8699 - acc: 0.3043 - mean_squared_error: 0.1193 - val_loss: 1.9689 - val_acc: 0.1429 - val_mean_squared_error: 0.1231

Epoch 00003: acc improved from 0.30435 to 0.30435, saving model to /home/willian/Keras_Litologia/model_save/Teste13/03_0.3013.json
Epoch 4/2500
46/46 [==============================] - 0s 947us

46/46 [==============================] - 0s 694us/step - loss: 1.0613 - acc: 0.5652 - mean_squared_error: 0.0775 - val_loss: 1.4467 - val_acc: 0.2857 - val_mean_squared_error: 0.0972

Epoch 00031: acc did not improve from 0.58696
Epoch 32/2500
46/46 [==============================] - 0s 616us/step - loss: 1.0277 - acc: 0.5870 - mean_squared_error: 0.0751 - val_loss: 1.4551 - val_acc: 0.4286 - val_mean_squared_error: 0.0993

Epoch 00032: acc did not improve from 0.58696
Epoch 33/2500
46/46 [==============================] - 0s 509us/step - loss: 1.0199 - acc: 0.6087 - mean_squared_error: 0.0751 - val_loss: 1.4506 - val_acc: 0.4286 - val_mean_squared_error: 0.0999

Epoch 00033: acc improved from 0.58696 to 0.60870, saving model to /home/willian/Keras_Litologia/model_save/Teste13/33_0.6113.json
Epoch 34/2500
46/46 [==============================] - 0s 746us/step - loss: 1.0009 - acc: 0.5652 - mean_squared_error: 0.0742 - val_loss: 1.3925 - val_acc: 0.5714 - val_mean_squared_error: 0.0952


Epoch 62/2500
46/46 [==============================] - 0s 479us/step - loss: 0.5930 - acc: 0.7391 - mean_squared_error: 0.0471 - val_loss: 0.6270 - val_acc: 0.8571 - val_mean_squared_error: 0.0450

Epoch 00062: acc did not improve from 0.76087
Epoch 63/2500
46/46 [==============================] - 0s 593us/step - loss: 0.5858 - acc: 0.7826 - mean_squared_error: 0.0468 - val_loss: 0.6080 - val_acc: 0.7143 - val_mean_squared_error: 0.0464

Epoch 00063: acc improved from 0.76087 to 0.78261, saving model to /home/willian/Keras_Litologia/model_save/Teste13/63_0.7813.json
Epoch 64/2500
46/46 [==============================] - 0s 454us/step - loss: 0.5781 - acc: 0.7391 - mean_squared_error: 0.0468 - val_loss: 0.5904 - val_acc: 0.7143 - val_mean_squared_error: 0.0444

Epoch 00064: acc did not improve from 0.78261
Epoch 65/2500
46/46 [==============================] - 0s 879us/step - loss: 0.5695 - acc: 0.6739 - mean_squared_error: 0.0461 - val_loss: 0.6046 - val_acc: 0.8571 - val_mean_squared_

46/46 [==============================] - 0s 859us/step - loss: 0.4663 - acc: 0.8043 - mean_squared_error: 0.0395 - val_loss: 0.3474 - val_acc: 0.8571 - val_mean_squared_error: 0.0279

Epoch 00094: acc did not improve from 0.84783
Epoch 95/2500
46/46 [==============================] - 0s 889us/step - loss: 0.4493 - acc: 0.7609 - mean_squared_error: 0.0393 - val_loss: 0.3073 - val_acc: 1.0000 - val_mean_squared_error: 0.0213

Epoch 00095: acc did not improve from 0.84783
Epoch 96/2500
46/46 [==============================] - 0s 584us/step - loss: 0.4348 - acc: 0.8261 - mean_squared_error: 0.0364 - val_loss: 0.3632 - val_acc: 0.8571 - val_mean_squared_error: 0.0301

Epoch 00096: acc did not improve from 0.84783
Epoch 97/2500
46/46 [==============================] - 0s 628us/step - loss: 0.4205 - acc: 0.8478 - mean_squared_error: 0.0348 - val_loss: 0.2953 - val_acc: 0.8571 - val_mean_squared_error: 0.0216

Epoch 00097: acc did not improve from 0.84783
Epoch 98/2500
46/46 [=================

46/46 [==============================] - 0s 895us/step - loss: 0.3405 - acc: 0.8478 - mean_squared_error: 0.0280 - val_loss: 0.2027 - val_acc: 0.8571 - val_mean_squared_error: 0.0146

Epoch 00127: acc did not improve from 0.89130
Epoch 128/2500
46/46 [==============================] - 0s 739us/step - loss: 0.3381 - acc: 0.8478 - mean_squared_error: 0.0280 - val_loss: 0.2373 - val_acc: 0.8571 - val_mean_squared_error: 0.0198

Epoch 00128: acc did not improve from 0.89130
Epoch 129/2500
46/46 [==============================] - 0s 735us/step - loss: 0.3396 - acc: 0.8478 - mean_squared_error: 0.0279 - val_loss: 0.1976 - val_acc: 0.8571 - val_mean_squared_error: 0.0140

Epoch 00129: acc did not improve from 0.89130
Epoch 130/2500
46/46 [==============================] - 0s 842us/step - loss: 0.3236 - acc: 0.8478 - mean_squared_error: 0.0265 - val_loss: 0.1799 - val_acc: 1.0000 - val_mean_squared_error: 0.0112

Epoch 00130: acc did not improve from 0.89130
Epoch 131/2500
46/46 [=============

...

46/46 [==============================] - 0s 589us/step - loss: 9.6871e-04 - acc: 1.0000 - mean_squared_error: 1.9288e-06 - val_loss: 2.3069 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02344: acc did not improve from 1.00000
Epoch 2345/2500
46/46 [==============================] - 0s 481us/step - loss: 9.9309e-04 - acc: 1.0000 - mean_squared_error: 2.1816e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02345: acc did not improve from 1.00000
Epoch 2346/2500
46/46 [==============================] - 0s 437us/step - loss: 9.9693e-04 - acc: 1.0000 - mean_squared_error: 2.0859e-06 - val_loss: 2.3066 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02346: acc did not improve from 1.00000
Epoch 2347/2500
46/46 [==============================] - 0s 462us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4316e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02347: acc did not improve from 1.00000
Epoc

46/46 [==============================] - 0s 522us/step - loss: 9.2168e-04 - acc: 1.0000 - mean_squared_error: 1.7042e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02376: acc did not improve from 1.00000
Epoch 2377/2500
46/46 [==============================] - 0s 504us/step - loss: 9.3679e-04 - acc: 1.0000 - mean_squared_error: 1.7444e-06 - val_loss: 2.3069 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02377: acc did not improve from 1.00000
Epoch 2378/2500
46/46 [==============================] - 0s 609us/step - loss: 9.8595e-04 - acc: 1.0000 - mean_squared_error: 2.1618e-06 - val_loss: 2.3069 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02378: acc did not improve from 1.00000
Epoch 2379/2500
46/46 [==============================] - 0s 606us/step - loss: 9.5541e-04 - acc: 1.0000 - mean_squared_error: 1.9928e-06 - val_loss: 2.3067 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02379: acc did not improve from 1.00000


46/46 [==============================] - 0s 545us/step - loss: 9.4044e-04 - acc: 1.0000 - mean_squared_error: 1.9075e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02408: acc did not improve from 1.00000
Epoch 2409/2500
46/46 [==============================] - 0s 761us/step - loss: 9.4612e-04 - acc: 1.0000 - mean_squared_error: 1.9847e-06 - val_loss: 2.3067 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02409: acc did not improve from 1.00000
Epoch 2410/2500
46/46 [==============================] - 0s 493us/step - loss: 9.1571e-04 - acc: 1.0000 - mean_squared_error: 1.8433e-06 - val_loss: 2.3065 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02410: acc did not improve from 1.00000
Epoch 2411/2500
46/46 [==============================] - 0s 451us/step - loss: 9.5008e-04 - acc: 1.0000 - mean_squared_error: 1.9769e-06 - val_loss: 2.3064 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02411: acc did not improve from 1.00000


46/46 [==============================] - 0s 551us/step - loss: 9.0853e-04 - acc: 1.0000 - mean_squared_error: 1.7144e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02440: acc did not improve from 1.00000
Epoch 2441/2500
46/46 [==============================] - 0s 489us/step - loss: 9.0559e-04 - acc: 1.0000 - mean_squared_error: 1.6300e-06 - val_loss: 2.3067 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02441: acc did not improve from 1.00000
Epoch 2442/2500
46/46 [==============================] - 0s 487us/step - loss: 9.1554e-04 - acc: 1.0000 - mean_squared_error: 1.7293e-06 - val_loss: 2.3066 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02442: acc did not improve from 1.00000
Epoch 2443/2500
46/46 [==============================] - 0s 910us/step - loss: 8.6373e-04 - acc: 1.0000 - mean_squared_error: 1.4855e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02443: acc did not improve from 1.00000


46/46 [==============================] - 0s 584us/step - loss: 9.0920e-04 - acc: 1.0000 - mean_squared_error: 1.7183e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02472: acc did not improve from 1.00000
Epoch 2473/2500
46/46 [==============================] - 0s 503us/step - loss: 8.7573e-04 - acc: 1.0000 - mean_squared_error: 1.5508e-06 - val_loss: 2.3068 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02473: acc did not improve from 1.00000
Epoch 2474/2500
46/46 [==============================] - 0s 537us/step - loss: 8.7387e-04 - acc: 1.0000 - mean_squared_error: 1.5604e-06 - val_loss: 2.3067 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02474: acc did not improve from 1.00000
Epoch 2475/2500
46/46 [==============================] - 0s 468us/step - loss: 8.6604e-04 - acc: 1.0000 - mean_squared_error: 1.4872e-06 - val_loss: 2.3066 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02475: acc did not improve from 1.00000


In [26]:
display_score(score_train13, score_test13)

 Baseline Error Train: 0.00%
 Train Accuracy: 1.0
 Baseline Error Test: 14.29%
 Test Accuracy: 0.8571428656578064


In [27]:
print(check_output(["ls", "/home/willian/Keras_Litologia/model_save/Teste13"]).decode("utf8"))

01_0.3013.json
03_0.3013.json
05_0.3313.json
06_0.3713.json
07_0.3913.json
08_0.5413.json
110_0.8913.json
144_0.9113.json
162_0.9313.json
197_0.9313.json
21_0.5713.json
219_0.9613.json
237_0.9813.json
26_0.5713.json
28_0.5913.json
306_1.0013.json
33_0.6113.json
35_0.6113.json
36_0.6313.json
37_0.6513.json
38_0.6713.json
47_0.7213.json
53_0.7413.json
57_0.7613.json
61_0.7613.json
63_0.7813.json
75_0.8013.json
76_0.8013.json
77_0.8313.json
89_0.8513.json



#### 1.4 Teste da Camada Oculta (Hidden Layer) = 20

In [28]:
model14, arquivo14 = verific_load_or_create(20, 20, 7, 'relu', 'relu', 'softmax', '14')

Created to model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_11 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_12 (Dense)             (None, 7)                 147       
Total params: 987
Trainable params: 987
Non-trainable params: 0
_________________________________________________________________


In [29]:
history14, score_train14, score_test14 = history_model(model14, arquivo14, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, '14')

Train on 46 samples, validate on 7 samples
Epoch 1/2500
46/46 [==============================] - 1s 17ms/step - loss: 1.9378 - acc: 0.2174 - mean_squared_error: 0.1221 - val_loss: 1.9370 - val_acc: 0.1429 - val_mean_squared_error: 0.1221

Epoch 00001: acc improved from -inf to 0.21739, saving model to /home/willian/Keras_Litologia/model_save/Teste14/01_0.2214.json
Epoch 2/2500
46/46 [==============================] - 0s 590us/step - loss: 1.9022 - acc: 0.2609 - mean_squared_error: 0.1206 - val_loss: 1.9241 - val_acc: 0.1429 - val_mean_squared_error: 0.1215

Epoch 00002: acc improved from 0.21739 to 0.26087, saving model to /home/willian/Keras_Litologia/model_save/Teste14/02_0.2614.json
Epoch 3/2500
46/46 [==============================] - 0s 672us/step - loss: 1.8580 - acc: 0.2174 - mean_squared_error: 0.1188 - val_loss: 1.9171 - val_acc: 0.1429 - val_mean_squared_error: 0.1212

Epoch 00003: acc did not improve from 0.26087
Epoch 4/2500
46/46 [==============================] - 0s 619us

Epoch 31/2500
46/46 [==============================] - 0s 719us/step - loss: 0.8202 - acc: 0.6739 - mean_squared_error: 0.0636 - val_loss: 1.0069 - val_acc: 0.5714 - val_mean_squared_error: 0.0734

Epoch 00031: acc did not improve from 0.67391
Epoch 32/2500
46/46 [==============================] - 0s 784us/step - loss: 0.7857 - acc: 0.6304 - mean_squared_error: 0.0610 - val_loss: 1.0659 - val_acc: 0.5714 - val_mean_squared_error: 0.0796

Epoch 00032: acc did not improve from 0.67391
Epoch 33/2500
46/46 [==============================] - 0s 511us/step - loss: 0.7647 - acc: 0.6522 - mean_squared_error: 0.0603 - val_loss: 1.0091 - val_acc: 0.5714 - val_mean_squared_error: 0.0759

Epoch 00033: acc did not improve from 0.67391
Epoch 34/2500
46/46 [==============================] - 0s 837us/step - loss: 0.7417 - acc: 0.6957 - mean_squared_error: 0.0584 - val_loss: 0.9498 - val_acc: 0.5714 - val_mean_squared_error: 0.0733

Epoch 00034: acc improved from 0.67391 to 0.69565, saving model to /ho

Epoch 63/2500
46/46 [==============================] - 0s 463us/step - loss: 0.4725 - acc: 0.7609 - mean_squared_error: 0.0376 - val_loss: 0.3821 - val_acc: 0.8571 - val_mean_squared_error: 0.0310

Epoch 00063: acc did not improve from 0.80435
Epoch 64/2500
46/46 [==============================] - 0s 621us/step - loss: 0.4628 - acc: 0.7826 - mean_squared_error: 0.0368 - val_loss: 0.3862 - val_acc: 0.8571 - val_mean_squared_error: 0.0306

Epoch 00064: acc did not improve from 0.80435
Epoch 65/2500
46/46 [==============================] - 0s 535us/step - loss: 0.4614 - acc: 0.7391 - mean_squared_error: 0.0367 - val_loss: 0.3392 - val_acc: 1.0000 - val_mean_squared_error: 0.0233

Epoch 00065: acc did not improve from 0.80435
Epoch 66/2500
46/46 [==============================] - 0s 749us/step - loss: 0.4623 - acc: 0.8043 - mean_squared_error: 0.0362 - val_loss: 0.3440 - val_acc: 0.8571 - val_mean_squared_error: 0.0260

Epoch 00066: acc did not improve from 0.80435
Epoch 67/2500
46/46 [===


Epoch 00095: acc did not improve from 0.82609
Epoch 96/2500
46/46 [==============================] - 0s 504us/step - loss: 0.3717 - acc: 0.8261 - mean_squared_error: 0.0294 - val_loss: 0.2088 - val_acc: 1.0000 - val_mean_squared_error: 0.0141

Epoch 00096: acc did not improve from 0.82609
Epoch 97/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.3763 - acc: 0.7826 - mean_squared_error: 0.0310 - val_loss: 0.2227 - val_acc: 1.0000 - val_mean_squared_error: 0.0154

Epoch 00097: acc did not improve from 0.82609
Epoch 98/2500
46/46 [==============================] - 0s 810us/step - loss: 0.3857 - acc: 0.8261 - mean_squared_error: 0.0315 - val_loss: 0.2301 - val_acc: 0.8571 - val_mean_squared_error: 0.0167

Epoch 00098: acc did not improve from 0.82609
Epoch 99/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.3659 - acc: 0.8478 - mean_squared_error: 0.0296 - val_loss: 0.2506 - val_acc: 0.8571 - val_mean_squared_error: 0.0202

Epoch 00099: acc improved f

46/46 [==============================] - 0s 538us/step - loss: 0.3472 - acc: 0.8261 - mean_squared_error: 0.0291 - val_loss: 0.1667 - val_acc: 1.0000 - val_mean_squared_error: 0.0101

Epoch 00128: acc did not improve from 0.89130
Epoch 129/2500
46/46 [==============================] - 0s 433us/step - loss: 0.3396 - acc: 0.8478 - mean_squared_error: 0.0276 - val_loss: 0.1823 - val_acc: 0.8571 - val_mean_squared_error: 0.0144

Epoch 00129: acc did not improve from 0.89130
Epoch 130/2500
46/46 [==============================] - 0s 540us/step - loss: 0.3790 - acc: 0.8478 - mean_squared_error: 0.0325 - val_loss: 0.2406 - val_acc: 0.8571 - val_mean_squared_error: 0.0227

Epoch 00130: acc did not improve from 0.89130
Epoch 131/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.3442 - acc: 0.8696 - mean_squared_error: 0.0282 - val_loss: 0.1470 - val_acc: 1.0000 - val_mean_squared_error: 0.0087

Epoch 00131: acc did not improve from 0.89130
Epoch 132/2500
46/46 [===============

...

46/46 [==============================] - 0s 753us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.7653e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02345: acc did not improve from 1.00000
Epoch 2346/2500
46/46 [==============================] - 0s 725us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.7698e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02346: acc did not improve from 1.00000
Epoch 2347/2500
46/46 [==============================] - 0s 509us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.7047e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02347: acc did not improve from 1.00000
Epoch 2348/2500
46/46 [==============================] - 0s 795us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5009e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02348: acc did not improve from 1.00000
Epoch 2349/2500


46/46 [==============================] - 0s 697us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4115e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02378: acc did not improve from 1.00000
Epoch 2379/2500
46/46 [==============================] - 0s 518us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4177e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02379: acc did not improve from 1.00000
Epoch 2380/2500
46/46 [==============================] - 0s 717us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4408e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02380: acc did not improve from 1.00000
Epoch 2381/2500
46/46 [==============================] - 0s 544us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5928e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02381: acc did not improve from 1.00000
Epoch 2382/2500


46/46 [==============================] - 0s 658us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.7179e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02411: acc did not improve from 1.00000
Epoch 2412/2500
46/46 [==============================] - 0s 685us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.8624e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02412: acc did not improve from 1.00000
Epoch 2413/2500
46/46 [==============================] - 0s 673us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4807e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02413: acc did not improve from 1.00000
Epoch 2414/2500
46/46 [==============================] - 0s 465us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.3866e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02414: acc did not improve from 1.00000
Epoch 2415/2500


46/46 [==============================] - 0s 684us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4362e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02444: acc did not improve from 1.00000
Epoch 2445/2500
46/46 [==============================] - 0s 705us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4445e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02445: acc did not improve from 1.00000
Epoch 2446/2500
46/46 [==============================] - 0s 548us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.5465e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02446: acc did not improve from 1.00000
Epoch 2447/2500
46/46 [==============================] - 0s 661us/step - loss: 9.9297e-04 - acc: 1.0000 - mean_squared_error: 1.7436e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02447: acc did not improve from 1.00000
Epoch 2448/2

46/46 [==============================] - 0s 644us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.4622e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02477: acc did not improve from 1.00000
Epoch 2478/2500
46/46 [==============================] - 0s 628us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.2111e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02478: acc did not improve from 1.00000
Epoch 2479/2500
46/46 [==============================] - 0s 884us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.5448e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02479: acc did not improve from 1.00000
Epoch 2480/2500
46/46 [==============================] - 0s 773us/step - loss: 0.0011 - acc: 1.0000 - mean_squared_error: 2.2700e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02480: acc did not improve from 1.00000
Epoch 2481/2500


In [30]:
display_score(score_train14, score_test14)

 Baseline Error Train: 0.00%
 Train Accuracy: 1.0
 Baseline Error Test: 14.29%
 Test Accuracy: 0.8571428656578064


In [31]:
print(check_output(["ls", "/home/willian/Keras_Litologia/model_save/Teste14"]).decode("utf8"))

01_0.2214.json
02_0.2614.json
07_0.3714.json
08_0.4614.json
09_0.5214.json
10_0.5414.json
11_0.5914.json
111_0.8514.json
112_0.8714.json
113_0.8914.json
138_0.9114.json
180_0.9114.json
18_0.6114.json
196_0.9314.json
217_0.9614.json
236_0.9614.json
24_0.6714.json
282_0.9814.json
30_0.6714.json
305_0.9814.json
34_0.7014.json
354_1.0014.json
39_0.7214.json
41_0.7614.json
57_0.7814.json
59_0.7814.json
62_0.8014.json
76_0.8314.json
79_0.8314.json
84_0.8314.json
99_0.8514.json



#### 1.5 Teste da Camada Oculta (Hidden Layer) = 27

In [32]:
model15, arquivo15 = verific_load_or_create(20, 27, 7, 'relu', 'relu', 'softmax', '15')

Created to model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_14 (Dense)             (None, 27)                567       
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 196       
Total params: 1,183
Trainable params: 1,183
Non-trainable params: 0
_________________________________________________________________


In [33]:
history15, score_train15, score_test15 = history_model(model15, arquivo15, x_train, y_train, x_test, y_test, epoch, batch_s, path_file_models, '15')

Train on 46 samples, validate on 7 samples
Epoch 1/2500
46/46 [==============================] - 1s 21ms/step - loss: 1.9439 - acc: 0.1739 - mean_squared_error: 0.1224 - val_loss: 1.9469 - val_acc: 0.1429 - val_mean_squared_error: 0.1225

Epoch 00001: acc improved from -inf to 0.17391, saving model to /home/willian/Keras_Litologia/model_save/Teste15/01_0.1715.json
Epoch 2/2500
46/46 [==============================] - 0s 651us/step - loss: 1.9243 - acc: 0.2174 - mean_squared_error: 0.1216 - val_loss: 1.9392 - val_acc: 0.1429 - val_mean_squared_error: 0.1222

Epoch 00002: acc improved from 0.17391 to 0.21739, saving model to /home/willian/Keras_Litologia/model_save/Teste15/02_0.2215.json
Epoch 3/2500
46/46 [==============================] - 0s 436us/step - loss: 1.8996 - acc: 0.1957 - mean_squared_error: 0.1205 - val_loss: 1.9326 - val_acc: 0.1429 - val_mean_squared_error: 0.1218

Epoch 00003: acc did not improve from 0.21739
Epoch 4/2500
46/46 [==============================] - 0s 603us

Epoch 30/2500
46/46 [==============================] - 0s 647us/step - loss: 0.8331 - acc: 0.6957 - mean_squared_error: 0.0624 - val_loss: 1.2513 - val_acc: 0.4286 - val_mean_squared_error: 0.0878

Epoch 00030: acc did not improve from 0.71739
Epoch 31/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.8281 - acc: 0.6739 - mean_squared_error: 0.0628 - val_loss: 1.2120 - val_acc: 0.4286 - val_mean_squared_error: 0.0868

Epoch 00031: acc did not improve from 0.71739
Epoch 32/2500
46/46 [==============================] - 0s 686us/step - loss: 0.8017 - acc: 0.6739 - mean_squared_error: 0.0610 - val_loss: 1.1559 - val_acc: 0.5714 - val_mean_squared_error: 0.0849

Epoch 00032: acc did not improve from 0.71739
Epoch 33/2500
46/46 [==============================] - 0s 776us/step - loss: 0.7810 - acc: 0.7174 - mean_squared_error: 0.0598 - val_loss: 1.1705 - val_acc: 0.5714 - val_mean_squared_error: 0.0881

Epoch 00033: acc improved from 0.71739 to 0.71739, saving model to /home


Epoch 00061: acc did not improve from 0.80435
Epoch 62/2500
46/46 [==============================] - 0s 585us/step - loss: 0.4701 - acc: 0.7826 - mean_squared_error: 0.0380 - val_loss: 0.5201 - val_acc: 0.7143 - val_mean_squared_error: 0.0449

Epoch 00062: acc did not improve from 0.80435
Epoch 63/2500
46/46 [==============================] - 0s 486us/step - loss: 0.4814 - acc: 0.7826 - mean_squared_error: 0.0394 - val_loss: 0.5088 - val_acc: 0.8571 - val_mean_squared_error: 0.0440

Epoch 00063: acc did not improve from 0.80435
Epoch 64/2500
46/46 [==============================] - 0s 683us/step - loss: 0.4661 - acc: 0.7826 - mean_squared_error: 0.0364 - val_loss: 0.4043 - val_acc: 0.8571 - val_mean_squared_error: 0.0324

Epoch 00064: acc did not improve from 0.80435
Epoch 65/2500
46/46 [==============================] - 0s 419us/step - loss: 0.4598 - acc: 0.8261 - mean_squared_error: 0.0354 - val_loss: 0.4130 - val_acc: 0.8571 - val_mean_squared_error: 0.0325

Epoch 00065: acc improv

46/46 [==============================] - 0s 490us/step - loss: 0.3768 - acc: 0.8043 - mean_squared_error: 0.0313 - val_loss: 0.2947 - val_acc: 0.8571 - val_mean_squared_error: 0.0225

Epoch 00094: acc did not improve from 0.89130
Epoch 95/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.3918 - acc: 0.8261 - mean_squared_error: 0.0322 - val_loss: 0.2722 - val_acc: 0.8571 - val_mean_squared_error: 0.0204

Epoch 00095: acc did not improve from 0.89130
Epoch 96/2500
46/46 [==============================] - 0s 733us/step - loss: 0.3981 - acc: 0.8478 - mean_squared_error: 0.0332 - val_loss: 0.3726 - val_acc: 0.8571 - val_mean_squared_error: 0.0312

Epoch 00096: acc did not improve from 0.89130
Epoch 97/2500
46/46 [==============================] - 0s 831us/step - loss: 0.3832 - acc: 0.8043 - mean_squared_error: 0.0327 - val_loss: 0.3283 - val_acc: 0.8571 - val_mean_squared_error: 0.0265

Epoch 00097: acc did not improve from 0.89130
Epoch 98/2500
46/46 [===================

46/46 [==============================] - 0s 577us/step - loss: 0.3291 - acc: 0.8696 - mean_squared_error: 0.0262 - val_loss: 0.2420 - val_acc: 0.8571 - val_mean_squared_error: 0.0187

Epoch 00127: acc did not improve from 0.95652
Epoch 128/2500
46/46 [==============================] - 0s 572us/step - loss: 0.3279 - acc: 0.8913 - mean_squared_error: 0.0273 - val_loss: 0.2605 - val_acc: 0.8571 - val_mean_squared_error: 0.0211

Epoch 00128: acc did not improve from 0.95652
Epoch 129/2500
46/46 [==============================] - 0s 523us/step - loss: 0.3121 - acc: 0.9130 - mean_squared_error: 0.0258 - val_loss: 0.2535 - val_acc: 0.8571 - val_mean_squared_error: 0.0208

Epoch 00129: acc did not improve from 0.95652
Epoch 130/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.3062 - acc: 0.8478 - mean_squared_error: 0.0246 - val_loss: 0.2754 - val_acc: 0.8571 - val_mean_squared_error: 0.0239

Epoch 00130: acc did not improve from 0.95652
Epoch 131/2500
46/46 [===============

...

46/46 [==============================] - 0s 459us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9372e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02342: acc did not improve from 1.00000
Epoch 2343/2500
46/46 [==============================] - 0s 583us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 3.1427e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02343: acc did not improve from 1.00000
Epoch 2344/2500
46/46 [==============================] - 0s 599us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9036e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02344: acc did not improve from 1.00000
Epoch 2345/2500
46/46 [==============================] - 0s 602us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9205e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02345: acc did not improve from 1.00000
Epoch 2346/2500


46/46 [==============================] - 0s 503us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9752e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02375: acc did not improve from 1.00000
Epoch 2376/2500
46/46 [==============================] - 0s 404us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.7134e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02376: acc did not improve from 1.00000
Epoch 2377/2500
46/46 [==============================] - 0s 611us/step - loss: 0.0014 - acc: 1.0000 - mean_squared_error: 3.3265e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02377: acc did not improve from 1.00000
Epoch 2378/2500
46/46 [==============================] - 0s 1ms/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9897e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02378: acc did not improve from 1.00000
Epoch 2379/2500
46

46/46 [==============================] - 0s 602us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.6913e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02408: acc did not improve from 1.00000
Epoch 2409/2500
46/46 [==============================] - 0s 640us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9536e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02409: acc did not improve from 1.00000
Epoch 2410/2500
46/46 [==============================] - 0s 683us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5024e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02410: acc did not improve from 1.00000
Epoch 2411/2500
46/46 [==============================] - 0s 543us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9225e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02411: acc did not improve from 1.00000
Epoch 2412/2500


46/46 [==============================] - 0s 937us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.4428e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02441: acc did not improve from 1.00000
Epoch 2442/2500
46/46 [==============================] - 0s 866us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.5025e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02442: acc did not improve from 1.00000
Epoch 2443/2500
46/46 [==============================] - 0s 750us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.4295e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02443: acc did not improve from 1.00000
Epoch 2444/2500
46/46 [==============================] - 0s 653us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.4089e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02444: acc did not improve from 1.00000
Epoch 2445/2500


46/46 [==============================] - 0s 537us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.1694e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02474: acc did not improve from 1.00000
Epoch 2475/2500
46/46 [==============================] - 0s 509us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 3.1198e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02475: acc did not improve from 1.00000
Epoch 2476/2500
46/46 [==============================] - 0s 489us/step - loss: 0.0012 - acc: 1.0000 - mean_squared_error: 2.3958e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02476: acc did not improve from 1.00000
Epoch 2477/2500
46/46 [==============================] - 0s 727us/step - loss: 0.0013 - acc: 1.0000 - mean_squared_error: 2.9263e-06 - val_loss: 2.3029 - val_acc: 0.8571 - val_mean_squared_error: 0.0408

Epoch 02477: acc did not improve from 1.00000
Epoch 2478/2500


In [34]:
display_score(score_train15, score_test15)

 Baseline Error Train: 0.00%
 Train Accuracy: 1.0
 Baseline Error Test: 14.29%
 Test Accuracy: 0.8571428656578064


In [35]:
print(check_output(["ls", "/home/willian/Keras_Litologia/model_save/Teste15"]).decode("utf8"))

01_0.1715.json
02_0.2215.json
08_0.2415.json
09_0.2615.json
10_0.3015.json
106_0.9615.json
11_0.4315.json
12_0.4615.json
14_0.5015.json
155_0.9615.json
18_0.5715.json
230_0.9815.json
23_0.6115.json
24_0.6315.json
26_0.6515.json
27_0.6515.json
285_1.0015.json
29_0.7215.json
33_0.7215.json
34_0.7215.json
37_0.7415.json
38_0.7815.json
56_0.7815.json
58_0.8015.json
65_0.8315.json
81_0.8715.json
91_0.8915.json

